In [1]:
import keepaAPI
import pickle
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
movie_asin = pd.read_excel('movie-asin.xlsx')
movie_asin.head()

,ID,PAR_ASINHC,PAR_ASINPB,PAR_ASINMM,PAR_UNIX_14,PAR_UNIX_28B,EXT_UNIX_90A
0,01M,1455582271,145558228X,NaN,1449100800,1450396800,1460678400
1,02M,193636574X,0307947513,NaN,1457568000,1458864000,1469145600
2,03M,0763655597,0763660655,NaN,1479945600,1481241600,1491523200
3,04M,NaN,0767902521,0307279464,1437436800,1438732800,1449014400
4,05M,1447273095,1548716332,NaN,1460592000,1461888000,1472169600


In [16]:
asinhc = list(movie_asin['PAR_ASINHC'].dropna())
asinpb = list(movie_asin['PAR_ASINPB'].dropna())
asinmm = list(list(movie_asin['PAR_ASINMM'].dropna()))

In [17]:
asinlist = asinhc + asinpb + asinmm

In [23]:
accesskey = "60jagklg8gepki5jnp34c3l2d8vb8f3qvphb8rdat5559nhbcb0birfo1n4bu6d1"
api = keepaAPI.API(accesskey)

INFO   : Connecting to keepa using key 60jagklg8gepki5jnp34c3l2d8vb8f3qvphb8rdat5559nhbcb0birfo1n4bu6d1
INFO   : 1200 tokens remain


In [24]:
movies_products = api.ProductQuery(asinlist)

INFO   : Executing 105 item product query
INFO   : Estimated time to complete 105 request(s) is 0.50 minutes
INFO   : 	with a refill rate of 20 token(s) per minute
INFO   : Completed 5 ASIN(s)
INFO   : Completed 105 ASIN(s)


In [3]:
with open("movie_product.pkl", "rb") as f:
    movies_products = pickle.load(f)

In [4]:
def get_date(x):
    return datetime.fromtimestamp((x+21564000)*60).strftime('%d-%m-%y')

def get_df(asin, unix_14, unix_28, unix_90, products207):
    for i in range(len(products207)):
        if products207[i]['asin'] == asin:
            product = products207[i]['csv'][3]
        
    date = [get_date(product[i]) for i in range(0,len(product),2)]
    unix_time = [(product[i]+21564000)*60 for i in range(0, len(product),2)]
    sale = [product[i] for i in range(1,len(product),2)]

    df = pd.DataFrame({'date': date, 'unix': unix_time, 'sale': sale})
    df = df.groupby('date', as_index = False)[['sale', 'unix']].mean()
    newdf = df[(df['unix'] > unix_28) & (df['unix'] < unix_90)]
    newdf = newdf.groupby('date', as_index = False)[['sale', 'unix']].mean()
    newdf.sort_values('unix', inplace = True)
    newdf.reset_index(inplace = True, drop = True)
    newdf1 = df[(df['unix'] > unix_14) & (df['unix'] < unix_28)]
    newdf1 = newdf1.groupby('date', as_index = False)[['sale', 'unix']].mean()
    newdf1.sort_values('unix', inplace = True)
    newdf1.reset_index(inplace = True, drop = True)
    return (df, newdf, newdf1)

In [27]:
df, newdf, newdf1 = get_df('0062339273', 1425513600, 1426809600, 1437091200, movies_products)

In [35]:
newdf.describe()

,sale,unix
count,91.000000,9.100000e+01
mean,42107.173090,1.432357e+09
std,46749.341324,2.860741e+06
min,1695.125000,1.427705e+09
25%,7303.166667,1.429818e+09
50%,29698.600000,1.432107e+09
75%,60062.750000,1.435081e+09
max,273118.000000,1.437046e+09


In [34]:
newdf['sale'].median()

29698.6

In [5]:
dfhc = movie_asin[movie_asin['PAR_ASINHC'].notnull()]
dfhc.drop(['PAR_ASINPB', 'PAR_ASINMM'], axis = 1, inplace = True)
dfhc.reset_index(drop = True, inplace = True)

dfpb = movie_asin[movie_asin['PAR_ASINPB'].notnull()]
dfpb.drop(['PAR_ASINHC', 'PAR_ASINMM'], axis = 1, inplace = True)
dfpb.reset_index(drop = True, inplace = True)

dfmm = movie_asin[movie_asin['PAR_ASINMM'].notnull()]
dfmm.drop(['PAR_ASINHC', 'PAR_ASINPB'], axis = 1, inplace = True)
dfmm.reset_index(drop = True, inplace = True)
dfmm.head()

/Users/hienn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/hienn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/hienn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,ID,PAR_ASINMM,PAR_UNIX_14,PAR_UNIX_28B,EXT_UNIX_90A
0,04M,0307279464,1437436800,1438732800,1449014400
1,10M,150112224X,1461801600,1463097600,1473379200
2,18M,1501140647,1451520000,1452816000,1463097600
3,22M,0440246326,1473292800,1474588800,1484870400
4,28M,1501147277,1464220800,1465516800,1475798400


In [6]:
resulthc = {}
for i, asin in enumerate(dfhc['PAR_ASINHC']):
    resulthc[dfhc['ID'][i]] = []
    df, newdf, newdf1 = get_df(asin, dfhc['PAR_UNIX_14'][i], dfhc['PAR_UNIX_28B'][i],
                      dfhc['EXT_UNIX_90A'][i], movies_products)
    resulthc[dfhc['ID'][i]].append(len(df))
    resulthc[dfhc['ID'][i]].append(len(newdf))
    resulthc[dfhc['ID'][i]].append(newdf['sale'].mean())
    resulthc[dfhc['ID'][i]].append(newdf['sale'].std())
    resulthc[dfhc['ID'][i]].append(newdf1['sale'].mean())
    
asinlist = []
total_dates = []
sale_date = []
mean_sale = []
std_sale = []
mean14 = []
for key, value in resulthc.items():
    asinlist.append(key)
    total_dates.append(value[0])
    sale_date.append(value[1])
    mean_sale.append(value[2])
    std_sale.append(value[3])
    mean14.append(value[4])
    
df2 = pd.DataFrame({'ID':asinlist, "HCTOTAL_DATES":total_dates, "HCSALE_DATES":sale_date,
                   "HCMEAN90": mean_sale, "HCSTD":std_sale, "HCMEAN14": mean14})
df2.head()

,HCMEAN14,HCMEAN90,HCSALE_DATES,HCSTD,HCTOTAL_DATES,ID
0,6557.395320,3005.999725,119,2124.219995,1027,01M
1,178834.899274,165269.624597,119,85517.768708,939,02M
2,13123.452381,18464.128365,104,13560.993695,629,03M
3,NaN,820708.893939,66,493841.312775,356,05M
4,75836.591366,108299.473298,119,55490.530803,950,07M


In [7]:
resultpb = {}
for i, asin in enumerate(dfpb['PAR_ASINPB']):
    resultpb[dfpb['ID'][i]] = []
    df, newdf, newdf1 = get_df(asin, dfpb['PAR_UNIX_14'][i], dfpb['PAR_UNIX_28B'][i],
                      dfpb['EXT_UNIX_90A'][i], movies_products)
    resultpb[dfpb['ID'][i]].append(len(df))
    resultpb[dfpb['ID'][i]].append(len(newdf))
    resultpb[dfpb['ID'][i]].append(newdf['sale'].mean())
    resultpb[dfpb['ID'][i]].append(newdf['sale'].std())
    resultpb[dfpb['ID'][i]].append(newdf1['sale'].mean())
    
asinlist = []
total_dates = []
sale_date = []
mean_sale = []
std_sale = []
mean14 = []
for key, value in resultpb.items():
    asinlist.append(key)
    total_dates.append(value[0])
    sale_date.append(value[1])
    mean_sale.append(value[2])
    std_sale.append(value[3])
    mean14.append(value[4])
    
df3 = pd.DataFrame({'ID':asinlist, "PBTOTAL_DATES":total_dates, "PBSALE_DATES":sale_date,
                   "PBMEAN90": mean_sale, "PBSTD":std_sale, "PBMEAN14": mean14})
df3.head()

,ID,PBMEAN14,PBMEAN90,PBSALE_DATES,PBSTD,PBTOTAL_DATES
0,01M,2153.484983,526.961749,119,397.144402,899
1,02M,26837.500000,22610.902778,60,11986.516305,716
2,03M,2326.350368,1972.866188,119,1013.595528,1039
3,04M,35504.876479,34424.274058,119,24911.463814,1007
4,05M,NaN,NaN,0,NaN,53


In [8]:
resultmm = {}
for i, asin in enumerate(dfmm['PAR_ASINMM']):
    resultmm[dfmm['ID'][i]] = []
    df, newdf, newdf1 = get_df(asin, dfmm['PAR_UNIX_14'][i], dfmm['PAR_UNIX_28B'][i],
                      dfmm['EXT_UNIX_90A'][i], movies_products)
    resultmm[dfmm['ID'][i]].append(len(df))
    resultmm[dfmm['ID'][i]].append(len(newdf))
    resultmm[dfmm['ID'][i]].append(newdf['sale'].mean())
    resultmm[dfmm['ID'][i]].append(newdf['sale'].std())
    resultmm[dfmm['ID'][i]].append(newdf1['sale'].mean())

asinlist = []
total_dates = []
sale_date = []
mean_sale = []
std_sale = []
mean14 = []
for key, value in resultmm.items():
    asinlist.append(key)
    total_dates.append(value[0])
    sale_date.append(value[1])
    mean_sale.append(value[2])
    std_sale.append(value[3])
    mean14.append(value[4])
    
df4 = pd.DataFrame({'ID':asinlist, "MMTOTAL_DATES":total_dates, "MMSALE_DATES":sale_date,
                   "MMMEAN90": mean_sale, "MMSTD":std_sale, "MMMEAN14": mean14})
df4.head()

,ID,MMMEAN14,MMMEAN90,MMSALE_DATES,MMSTD,MMTOTAL_DATES
0,04M,566.973572,515.504411,119,369.357592,1012
1,10M,NaN,37974.558316,46,21049.908952,343
2,18M,NaN,18633.297681,117,16086.447967,388
3,22M,32896.166667,46591.184524,56,28072.065833,780
4,28M,NaN,185383.294048,42,164237.174079,231


In [10]:
df6 = df2.merge(df3, on = 'ID', how = 'outer')
len(df6)

62

In [11]:
df7 = df6.merge(df4, on = 'ID', how = 'outer')
len(df7)

63

In [12]:
newfile = df7.merge(movie_asin, on = 'ID', how = 'outer')

In [15]:
newfile.to_excel('movie_fullver.xlsx')

In [17]:
newfile.head()

,HCMEAN14,HCMEAN90,HCSALE_DATES,HCSTD,HCTOTAL_DATES,ID,PBMEAN14,PBMEAN90,PBSALE_DATES,PBSTD,...,MMMEAN90,MMSALE_DATES,MMSTD,MMTOTAL_DATES,PAR_ASINHC,PAR_ASINPB,PAR_ASINMM,PAR_UNIX_14,PAR_UNIX_28B,EXT_UNIX_90A
0,6557.395320,3005.999725,119.0,2124.219995,1027.0,01M,2153.484983,526.961749,119.0,397.144402,...,NaN,NaN,NaN,NaN,1455582271,145558228X,NaN,1449100800,1450396800,1460678400
1,178834.899274,165269.624597,119.0,85517.768708,939.0,02M,26837.500000,22610.902778,60.0,11986.516305,...,NaN,NaN,NaN,NaN,193636574X,0307947513,NaN,1457568000,1458864000,1469145600
2,13123.452381,18464.128365,104.0,13560.993695,629.0,03M,2326.350368,1972.866188,119.0,1013.595528,...,NaN,NaN,NaN,NaN,0763655597,0763660655,NaN,1479945600,1481241600,1491523200
3,NaN,820708.893939,66.0,493841.312775,356.0,05M,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,1447273095,1548716332,NaN,1460592000,1461888000,1472169600
4,75836.591366,108299.473298,119.0,55490.530803,950.0,07M,18399.595556,53314.443071,106.0,36666.367743,...,NaN,NaN,NaN,NaN,0060885599,0060885610,NaN,1475712000,1477008000,1487289600


In [18]:
newfile.drop(['PAR_UNIX_14', 'PAR_UNIX_28B', 'EXT_UNIX_90A'], axis = 1, inplace = True)

In [16]:
movie_sequel = pd.read_excel('movie-sequel.xlsx')
movie_sequel.head()

,ID,EXT_TITLE,EXT_DATE,EXT_DATE90A,EXT_UNIX_90A,EXT_DATE28B,PAR_UNIX_28B,EXT_DATE14,PAR_UNIX14,full-title,...,ID.1,PBMEAN14,PBMEAN90,PBSTD,PBTOTAL_DATES,MMMEAN14,MMMEAN90,MMSALE_DATES,MMSTD,MMTOTAL_DATES
0,01M,13 Hours,2016-01-15,2016-04-15,1.460678e+09,2015-12-18,1.450397e+09,2015-12-03,1.449101e+09,13 Hours: The Inside Account of What Really Ha...,...,25M,1.845192e+03,1.430171e+03,695.653682,848.0,NaN,NaN,NaN,NaN,NaN
1,02M,A Hologram for the King,2016-04-22,2016-07-22,1.469146e+09,2016-03-25,1.458864e+09,2016-03-10,1.457568e+09,A Hologram for the King: A No...,...,58M,3.019912e+02,6.184918e+02,287.522300,1054.0,NaN,NaN,NaN,NaN,NaN
2,03M,A Monster Calls,2017-01-06,2017-04-07,1.491523e+09,2016-12-09,1.481242e+09,2016-11-24,1.479946e+09,A Monster Calls: Inspired by an idea from Siob...,...,11M,5.183333e+03,6.498932e+03,3746.174347,934.0,NaN,NaN,NaN,NaN,NaN
3,04M,A Walk In the Woods,2015-09-02,2015-12-02,1.449014e+09,2015-08-05,1.438733e+09,2015-07-21,1.437437e+09,A Walk in the Woods: Rediscovering America on ...,...,17M,NaN,1.796069e+02,153.671647,1042.0,NaN,NaN,NaN,NaN,NaN
4,05M,Alice Through the Looking-Glass,2016-05-27,2016-08-26,1.472170e+09,2016-04-29,1.461888e+09,2016-04-14,1.460592e+09,Through The Looking-Glass,...,61M,2.126106e+06,1.732171e+06,517961.177731,326.0,215194.972222,116332.405827,92.0,100611.33859,669.0


In [19]:
newfile2 = movie_sequel.merge(newfile, on = 'ID', how = 'outer')

In [20]:
newfile2.to_excel('movie-fullver2.xlsx')

In [21]:
df2

,HCMEAN14,HCMEAN90,HCSALE_DATES,HCSTD,HCTOTAL_DATES,ID
0,6.557395e+03,3.006000e+03,119,2.124220e+03,1027,01M
1,1.788349e+05,1.652696e+05,119,8.551777e+04,939,02M
2,1.312345e+04,1.846413e+04,104,1.356099e+04,629,03M
3,NaN,8.207089e+05,66,4.938413e+05,356,05M
4,7.583659e+04,1.082995e+05,119,5.549053e+04,950,07M
5,8.012398e+05,4.994143e+05,102,2.973206e+05,402,09M
6,2.810500e+04,8.908793e+04,15,5.763142e+04,465,11M
7,9.868998e+01,5.464790e+01,102,5.721503e+01,604,14M
8,NaN,2.054316e+04,52,1.374802e+04,474,15M
9,4.600944e+05,2.421818e+05,59,1.796742e+05,738,16M


In [2]:
curr = pd.read_excel('movie-sequel1.xlsx')
curr

,ID,EXT_TITLE,EXT_DATE,EXT_DATE90A,EXT_UNIX_90A,EXT_DATE28B,PAR_UNIX_28B,EXT_DATE14,PAR_UNIX14,full-title,...,PBTOTAL_DATES,PB_dates,PAR_ASINMM,MMMEAN14,MMMEAN90,MMSALE_DATES,MMSTD,MMTOTAL_DATES,mm_dates,Original published date
0,17M,Fifty Shades of Grey,2015-02-13,2015-05-15,1.431648e+09,2015-01-16,1.421366e+09,2015-01-01,1.420070e+09,Fifty Shades of Grey: Book One of the Fifty Sh...,...,1042.0,2012-04-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,44M,The DUFF,2015-02-20,2015-05-22,1.432253e+09,2015-01-23,1.421971e+09,2015-01-08,1.420675e+09,The DUFF: (Designated Ugly Fat Frie...,...,915.0,2011-06-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,58M,Insurgent,2015-03-20,2015-06-19,1.434672e+09,2015-02-20,1.424390e+09,2015-02-05,1.423094e+09,Insurgent (Divergent Seri...,...,1054.0,2015-01-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,49M,The Longest Ride,2015-04-10,2015-07-10,1.436486e+09,2015-03-13,1.426205e+09,2015-02-26,1.424909e+09,NaN,...,487.0,2014-05-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,55M,True Story,2015-04-17,2015-07-17,1.437091e+09,2015-03-20,1.426810e+09,2015-03-05,1.425514e+09,NaN,...,746.0,2015-04-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
5,11M,Child 44,2015-04-17,2015-07-17,1.437091e+09,2015-03-20,1.426810e+09,2015-03-05,1.425514e+09,Child 44 (The Child 44 Trilo...,...,934.0,2011-12-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
6,15M,Far from the Madding Crowd,2015-05-22,2015-08-21,1.440115e+09,2015-04-24,1.429834e+09,2015-04-09,1.428538e+09,Far from the Madding Crowd by Thomas Hardy Una...,...,151.0,2017-09-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
7,12M,Dark Places,2015-06-18,2015-09-17,1.442448e+09,2015-05-21,1.432166e+09,2015-05-06,1.430870e+09,Dark Pla...,...,1043.0,2010-05-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
8,25M,Me and Earl and the Dying Girl,2015-07-01,2015-09-30,1.443571e+09,2015-06-03,1.433290e+09,2015-05-19,1.431994e+09,Me and Earl and the Dying Girl (Revised Editi...,...,848.0,2015-04-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
9,33M,Paper Towns,2015-07-24,2015-10-23,1.445558e+09,2015-06-26,1.435277e+09,2015-06-11,1.433981e+09,Paper To...,...,1045.0,2009-09-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [5]:
len(curr)

66

In [3]:
new = curr[(curr['PBSALE_DATES']>89) | (curr['HCSALE_DATES'] >89) | (curr['MMSALE_DATES'] > 89)]

In [6]:
len(new)

47

In [26]:
new.to_excel('movie-cutdown.xlsx')

In [7]:
df1 = pd.read_excel('cutdown-ver1-current.xlsx')
df2 = pd.read_excel('file2merge.xlsx')

In [8]:
len(df1)

144

In [9]:
len(df2)

316

In [10]:
newfile = df1.merge(df2, on = 'ID')
len(newfile)

144

In [11]:
newfile.to_excel('book-ver1.xlsx')

In [2]:
df = pd.read_excel('cutdown-ver5.xlsx')
df.head()

,EXT_FORMAT,ID,EXT_TITLE,EXT_ORDER,EXT_DATE,EXT_DATE_90A,Unnamed: 5,EXT_UNIX_90A_y,EXT_RATING,EXT_VOLUME,...,PAR_UNIX_14_y,PAR_RATING_28B,PAR_VOLUME_28B,MEAN14,MEAN90,SALE_DATES,STD,TOTAL_DATES,PAR_ASIN,FORMAT
0,B,1424,The Heir (The Selection #4),4,2015-05-05,2015-08-03,2015-08-04,1438646400,4.1,837.0,...,1427068800,4.5,2818.0,1878.719529,1126.092273,111,484.187421,1016,0062059947,PB
1,B,932,Hidden Huntress (The Malediction Trilogy #2),2,2015-06-02,2015-08-31,2015-09-01,1441065600,4.3,87.0,...,1429488000,4.4,283.0,40973.988004,52836.026036,103,27982.847787,947,1908844965,PB
2,B,2032,The Invasion of the Tearling (The Queen of the...,2,2015-06-09,2015-09-07,2015-09-08,1441670400,4.4,154.0,...,1430092800,4.2,354.0,5726.625000,7802.926785,105,5610.917463,962,006229038X,PB
3,B,984,Grey (Fifty Shades #4),4,2015-06-18,2015-09-16,2015-09-17,1442448000,4.2,10220.0,...,1430870400,4.0,73397.0,181.914059,727.993802,112,482.940459,1042,0345803485,PB
4,B,292,The Philosopher Kings (Thessaly #2),2,2015-06-30,2015-09-28,2015-09-29,1443484800,NaN,NaN,...,1431907200,NaN,NaN,84621.500000,120890.634048,90,94338.690878,819,0765332663,HC


In [3]:
df['EXT_ORDER'].value_counts()

2     62
3     44
5     15
4     15
6      3
7      2
10     1
9      1
8      1
Name: EXT_ORDER, dtype: int64

In [5]:
15+15+3+2+1+1+1

38